In [1]:
import pandas as pd 
import requests
from IPython.display import clear_output
import os

sep = os.sep
current_path = os.getcwd()
splitted_path = current_path.split(sep)
DATA_PATH = os.path.join(sep.join(splitted_path[:-2]), "data")

os.path.exists(DATA_PATH)

True

In [4]:
data = pd.read_csv(f'{DATA_PATH}/raw/Procesos_en_Casas_de_Justicia_20241103.csv', engine='python')
data

,IdSolicitud,NomEntidad,NomCasaJusticia,FechaSolicitud
0,481137,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,10/21/2019
1,3930874,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-05-08T00:00:00.000
2,414466,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,09/16/2019
3,441614,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,09/30/2019
4,39309,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-05-08T00:00:00.000
...,...,...,...,...
3737187,6998858,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 18:32:12.7903092
3737188,6998861,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 18:38:10.1276910
3737189,6998863,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 18:59:42.8782731
3737190,6998867,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 20:03:26.6302227


In [9]:
data.isna().sum()

IdSolicitud           0
NomEntidad         1334
NomCasaJusticia       0
FechaSolicitud        0
dtype: int64

## Include longitude and latitude

In [10]:
data['NomCasaJusticia'].unique()

array(['Casa de Justicia Cajicá', 'Casa de Justicia Frontino',
       'Casa de Justicia Tocancipa', 'Casa de Justicia Chía',
       'Casa de Justicia Caucasia', 'Casa de Justicia Yarumal',
       'Casa de Justicia Corinto', 'Casa de Justicia Floridablanca',
       'Centro de Convivencia Ciudadana Viota', 'Casa de Justicia Amalfí',
       'Casa de Justicia Manizales', 'Casa de Justicia Soacha',
       'Centro de Convivencia Ciudadana Paipa',
       'Casa de Justicia La Ceja', 'Casa de Justicia Pereira Cuba',
       'Centro de Convivencia Ciudadana Sonsón',
       'Centro de Convivencia Ciudadana Espinal',
       'Casa de justicia Cali Aguablanca',
       'Centro de Convivencia Ciudadana Chiquinquira',
       'Casa de Justicia Santa fe de Antioquía',
       'Casa de justicia Cali Siloé',
       'Casa de justicia Barranquilla Simón Bolívar',
       'Casa de Justicia Neiva', 'Casa de Justicia Supía',
       'Casa de Justicia Montería', 'Casa de Justicia Girardot',
       'Casa de Justicia 

In [11]:
def get_coordinates(place_name: str) -> tuple[float, float] | None:
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': str(place_name),      
        'format': 'json',      
        'addressdetails': 1,   
        'limit': 1             
    }

    headers = {
        'User-Agent': 'TuApp/1.0 (tucorreo@dominio.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return lat, lon    

In [12]:
row = data['NomCasaJusticia'].unique()

for index, place in enumerate(row):
    clear_output(wait=True)

    display(f'{index + 1} / {len(row)}')

    response = get_coordinates(place_name=place)
    
    if not response:
        data = data.drop(data[data['NomCasaJusticia'] == place].index)
        continue

    lat, lon = response
    data.loc[data['NomCasaJusticia'] == place, ['lat', 'lon']] = lat, lon

data

'134 / 134'

,IdSolicitud,NomEntidad,NomCasaJusticia,FechaSolicitud,lat,lon
0,481137,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,10/21/2019,4.9178877,-74.02831549556014
1,3930874,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-05-08T00:00:00.000,4.9178877,-74.02831549556014
2,414466,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,09/16/2019,4.9178877,-74.02831549556014
3,441614,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,09/30/2019,4.9178877,-74.02831549556014
4,39309,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-05-08T00:00:00.000,4.9178877,-74.02831549556014
...,...,...,...,...,...,...
3737187,6998858,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 18:32:12.7903092,11.22549585,-74.20498321082647
3737188,6998861,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 18:38:10.1276910,11.22549585,-74.20498321082647
3737189,6998863,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 18:59:42.8782731,11.22549585,-74.20498321082647
3737190,6998867,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30 20:03:26.6302227,11.22549585,-74.20498321082647


In [13]:
data.isna().sum()

IdSolicitud          0
NomEntidad         428
NomCasaJusticia      0
FechaSolicitud       0
lat                  0
lon                  0
dtype: int64

## Fix date format

In [27]:
data['FechaSolicitud']

0                           10/21/2019
1              2019-05-08T00:00:00.000
2                           09/16/2019
3                           09/30/2019
4              2019-05-08T00:00:00.000
                      ...             
3737187    2024-09-30 18:32:12.7903092
3737188    2024-09-30 18:38:10.1276910
3737189    2024-09-30 18:59:42.8782731
3737190    2024-09-30 20:03:26.6302227
3737191    2024-09-30 20:06:36.7694749
Name: FechaSolicitud, Length: 1225489, dtype: object

In [ ]:
def date_convert(fecha):
    formatos = [
        "%m/%d/%Y",         # formato MM/DD/YYYY
        "%Y-%m-%dT%H:%M:%S.%f",  # formato ISO con milisegundos
        "%Y-%m-%d %H:%M:%S.%f",  # formato con milisegundos y espacios
        "%Y-%m-%d",         # formato YYYY-MM-DD
        "%m/%d/%Y %H:%M:%S", # formato MM/DD/YYYY HH:MM:SS
    ]
    
    for fmt in formatos:
        try:
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            continue
    return pd.NaT


data['FechaSolicitud'] = data['FechaSolicitud'].apply(date_convert)
data['FechaSolicitud'] = data['FechaSolicitud'].dt.strftime('%Y-%m-%d')
fechas_invalidas = data['FechaSolicitud'].isna().sum()
print(f"Fechas no válidas encontradas después del intento de conversión: {fechas_invalidas}")


Fechas no válidas encontradas después del intento de conversión: 0


In [30]:
data

,IdSolicitud,NomEntidad,NomCasaJusticia,FechaSolicitud,lat,lon
0,481137,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-10-21,4.9178877,-74.02831549556014
1,3930874,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-05-08,4.9178877,-74.02831549556014
2,414466,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-09-16,4.9178877,-74.02831549556014
3,441614,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-09-30,4.9178877,-74.02831549556014
4,39309,UNIVERSIDAD FACULTAD DE DERECHO CONSULTORIOJ...,Casa de Justicia Cajicá,2019-05-08,4.9178877,-74.02831549556014
...,...,...,...,...,...,...
3737187,6998858,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30,11.22549585,-74.20498321082647
3737188,6998861,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30,11.22549585,-74.20498321082647
3737189,6998863,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30,11.22549585,-74.20498321082647
3737190,6998867,Conciliación en Equidad,Casa de Justicia Santa Marta,2024-09-30,11.22549585,-74.20498321082647


In [31]:
data.to_csv(f'{DATA_PATH}/processed/Procesos_casas_de_justicia.csv', index=False)